In [ ]:
const { postTweet } = require('./twitterService');

async function postThread({ uid, tweets, contentId }) {
    if (!tweets || !Array.isArray(tweets) || tweets.length === 0) throw new Error("No tweets to post");
    let lastTweetId = null;
    const postedTweets = [];

    for (let i = 0; i < tweets.length; i++) {
        const text = tweets[i];
        // Only associate the contentId with the FIRST tweet of the thread to avoid duplicate tracking entries?
        // Or maybe strictly speaking we want the trace. Let's pass it to the first one mainly.
        const res = await postTweet({
            uid,
            text,
            contentId: i === 0 ? contentId : undefined,
            replyToTweetId: lastTweetId
        });
        if (!res || !res.tweetId) throw new Error("Failed to post tweet in thread chain");
        lastTweetId = res.tweetId;
        postedTweets.push(res);
    }
    
    return {
        success: true,
        platform: "twitter",
        threadId: postedTweets[0].tweetId,
        length: postedTweets.length,
        tweets: postedTweets
    };
}